In [1]:
import json, requests
import string
import re
import gzip
import shutil
import urllib.request 
import pandas as pd
# import imdbpy


# name gender num_times_nominated age_nomination age_when_win

oscars = pd.read_csv("../data/the_oscar_award_corrected.csv")
imdb = pd.read_csv("../data/namebasics.tsv", sep='\t')
films = pd.read_csv("../data/oracle_of_bacon_data/clean_bacon_df.csv")
films

,title,year,role,name
0,The Birth of a Nation,1915,star,Lillian Gish
1,The Birth of a Nation,1915,cast,Donald Crisp
2,Blade Runner,1982,star,Harrison Ford
3,Blade Runner,1982,star,Edward James Olmos
4,Blade Runner,1982,director,Ridley Scott
...,...,...,...,...
46108,Island (2011 film),2011,cast,Janet McTeer
46109,Shadow Over Elveron,1968,cast,Shirley Knight
46110,Shadow Over Elveron,1968,cast,Franchot Tone
46111,Shadow Over Elveron,1968,cast,Don Ameche


In [2]:
########## Adding gender column

oscars['gender'] = ''
imdb.loc[imdb['primaryProfession'].str.contains("actor",na=False),  "gender"]='M'
imdb.loc[imdb['primaryProfession'].str.contains("actress",na=False), "gender"]='F'
# iFlowStatus = Sales[Sales['Product'].str.contains('iFlow', na=False)]['Status']

imdb.head()
gend_name = imdb[['gender', 'primaryName']]
gend_name.head()
gend_name=gend_name.rename(columns={"primaryName":"name"})
gend_name.head()


,gender,name
0,M,Fred Astaire
1,F,Lauren Bacall
2,F,Brigitte Bardot
3,M,John Belushi
4,M,Ingmar Bergman


In [15]:
years = pd.merge(gend_name, films, how='inner', on=['name'])
# years_grouped = years.groupby(['gender', 'year']).count().reset_index()
# nominations = nominees.groupby(['name','winner_x', 'gender_x', 'birthYear', 'year_film','category_x']).count().reset_index()

# years_grouped.tail(100)
imdb = imdb.rename(columns={"primaryName": "name"})
all_years = pd.merge(years, imdb, how='inner', on=['name'])
all_years = all_years.replace(r'\\N', 0, regex=True)
# all_years = all_years.dropna('birthYear')
# all_years = all_years.dropna(axis=0, subset=['primaryProfession'])
# all_years = all_years.dropna(axis=0, subset=['primaryProfession'])
# indexNames = all_years[all_years['birthYear'] == 0].index
 
# Delete these row indexes from dataFrame





In [72]:
all_years2 = all_years
all_years2 = all_years2.drop(all_years[all_years['birthYear'] == 0].index ).reset_index()
# all_years2 = all_years2.drop(all_years[pd.to_numeric(all_years['year']) > pd.to_numeric(all_years['deathYear'])].index )
all_years2 = all_years2.drop(['index', 'gender_y',  'knownForTitles'], axis=1)

all_years2[["birthYear", "deathYear", "year"]] = all_years2[["birthYear", "deathYear", "year"]].apply(pd.to_numeric)
all_years2["roleAge"] = all_years2['year']-all_years2['birthYear']
# all_years2["roleAge"]=pd.to_numeric(all_years2['year'], errors='coerce')-pd.to_numeric(all_years2['birthYear'], errors='coerce') 
all_years2["posthumous"] = False
all_years2.loc[ (all_years2.year>all_years2.deathYear), 'posthumous'] = True 

all_years2 = all_years2.dropna()
# all_years2["roleAge"] = all_years2['year']-all_years2['birthYear']
all_years2.head(50)



# Dropping all posthumous roles
posthumous = all_years2[all_years2['posthumous'] == True]  # Saving posthumous roles in new dataset
all_years2 = all_years2.drop(posthumous.index ).reset_index()

all_years2 = all_years2.drop(['index'], axis=1)
all_years2.to_csv('../data/all_acting_roles.csv', index=False)
posthumous.to_csv('../data/posthumous_roles.csv', index=False)
all_years2.head(50)




,gender_x,name,title,year,role,nconst,birthYear,deathYear,primaryProfession,roleAge,posthumous
0,M,Fred Astaire,The Gay Divorcee,1934,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",35,False
1,M,Fred Astaire,Top Hat,1935,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",36,False
2,M,Fred Astaire,The Band Wagon,1953,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",54,False
3,M,Fred Astaire,The Towering Inferno,1974,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",75,False
4,M,Fred Astaire,Holiday Inn (film),1942,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",43,False
5,M,Fred Astaire,The Story of Vernon and Irene Castle,1939,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",40,False
6,M,Fred Astaire,Funny Face,1957,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",58,False
7,M,Fred Astaire,Easter Parade (film),1948,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",49,False
8,M,Fred Astaire,Swing Time (film),1936,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",37,False
9,M,Fred Astaire,Blue Skies (1946 film),1946,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",47,False


In [73]:
stars = all_years2[all_years2['role']=='star']
stars.to_csv('../data/starring_roles.csv', index=False)
all
stars.head(50)

,gender_x,name,title,year,role,nconst,birthYear,deathYear,primaryProfession,roleAge,posthumous
0,M,Fred Astaire,The Gay Divorcee,1934,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",35,False
1,M,Fred Astaire,Top Hat,1935,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",36,False
2,M,Fred Astaire,The Band Wagon,1953,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",54,False
3,M,Fred Astaire,The Towering Inferno,1974,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",75,False
4,M,Fred Astaire,Holiday Inn (film),1942,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",43,False
5,M,Fred Astaire,The Story of Vernon and Irene Castle,1939,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",40,False
6,M,Fred Astaire,Funny Face,1957,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",58,False
7,M,Fred Astaire,Easter Parade (film),1948,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",49,False
8,M,Fred Astaire,Swing Time (film),1936,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",37,False
9,M,Fred Astaire,Blue Skies (1946 film),1946,star,nm0000001,1899,1987,"soundtrack,actor,miscellaneous",47,False


In [70]:
stars.groupby(['gender_x','year']).count().tail(50)

name  title  role  nconst  birthYear  deathYear  \
gender_x year                                                    
M        1971   369    369   369     369        369        369   
         1972   233    233   233     233        233        233   
         1973   335    335   335     335        335        335   
         1974   270    270   270     270        270        270   
         1975   258    258   258     258        258        258   
         1976   317    317   317     317        317        317   
         1977   231    231   231     231        231        231   
         1978   327    327   327     327        327        327   
         1979   328    328   328     328        328        328   
         1980   278    278   278     278        278        278   
         1981   217    217   217     217        217        217   
         1982   186    186   186     186        186        186   
         1983   172    172   172     172        172        172   
         1984   155    155   155     155        155        155   
         1985   151    151   151     151        151        151   
         1986   157    157   157     157        157        157   
         1987   128    128   128     128        128        128   
         1988   171    171   171     171        171        171   
         1989   212    212   212     212        212        212   
         1990   162    162   162     162        162        162   
         1991   171    171   171     171        171        171   
         1992   126    126   126     126        126        126   
         1993   176    176   176     176        176        176   
         1994   158    158   158     158        158        158   
         1995   142    142   142     142        142        142   
         1996   183    183   183     183        183        183   
         1997   187    187   187     187        187        187   
         1998   156    156   156     156        156        156   
         1999   157    157   157     157        157        157   
         2000   161    161   161     161        161        161   
         2001   146    146   146     146        146        146   
         2002   156    156   156     156        156        156   
         2003   125    125   125     125        125        125   
         2004    96     96    96      96         96         96   
         2005   100    100   100     100        100        100   
         2006   191    191   191     191        191        191   
         2007    93     93    93      93         93         93   
         2008   132    132   132     132        132        132   
         2009   164    164   164     164        164        164   
         2010    89     89    89      89         89         89   
         2011    48     48    48      48         48         48   
         2012    38     38    38      38         38         38   
         2013    38     38    38      38         38         38   
         2014    52     52    52      52         52         52   
         2015    19     19    19      19         19         19   
         2016    21     21    21      21         21         21   
         2017    29     29    29      29         29         29   
         2018    15     15    15      15         15         15   
         2019    10     10    10      10         10         10   
         2020     1      1     1       1          1          1   

               primaryProfession  roleAge  posthumous  
gender_x year                                          
M        1971                369      369         369  
         1972                233      233         233  
         1973                335      335         335  
         1974                270      270         270  
         1975                258      258         258  
         1976                317      317         317  
         1977                231      231         231  
         1978                327      327         327  
         1979         